RA work for Professor Albert Youngil Kim 

Title : Worker productivity and recognition
Work Description : Gathering information about Oscar nominees and their filmographies from Wikipedia and IMDB using an Application Programming Interface (API)

In [16]:
import pandas as pd
from imdb import IMDb
import time
from imdb import Cinemagoer
import imdb

In [ ]:
df_list = []
for i in range(2, 13):
    url= 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor#Winners_and_nominees'
    df_list.append(pd.read_html(url)[i])

df = pd.concat(df_list)

In [ ]:
df['id'] = 0

In [ ]:
for actor in df.Actor.loc[df.id==0]:
    try: 
        search = ia.search_person(actor)
        name_id = search[0].personID
        df.id.loc[df.Actor==actor] = name_id
    except IndexError:
        pass

In [ ]:
len(df.Actor.loc[df.id==0]) 

In [ ]:
df['id'] = 'nm' + df['id'].astype(str)

In [ ]:
df.to_csv("best_actor.csv", index = False)

In [17]:
df = pd.read_csv('best_actor.csv', index_col=0)
df_index = df.set_index(df['id'])
df_index = df_index.loc[~df_index.index.duplicated(keep='first')]
df_index.index.name = 'id_'
df_index.reset_index(inplace=True)
df_index.drop(columns=['id_'], inplace=True)

In [ ]:
moviesDB = imdb.IMDb()
ia = Cinemagoer() 

out = pd.DataFrame(columns=['Name', 'Birthdate', 'Deathdate', 'Title', 'Type', 'Year', 'Award Year', \
                            'Oscar', 'Cannes', 'Berlin', 'Venice', 'Golden Globe', 'IMDB Rating', \
                            'Actor', 'Producer', 'Director', 'Writer', 'Budget', 'Box Office', 'Running Time'])

j = 0
for p, id_ in zip(df_index['Actor'], df_index['id']):
    ActorId = id_[2:]
    person = moviesDB.get_person(ActorId)   
    actor_work = moviesDB.get_person_filmography(ActorId)
    
    try:
        actor_work['data']['filmography']
    
    except KeyError:
        time.sleep(30)
        actor_work = moviesDB.get_person_filmography(ActorId)

        
    for movie in actor_work['data']['filmography']['actor']:
        movie_id = movie.movieID
        tmp = ia.get_movie(movie_id)
        out.loc[j, 'Name'] = p
        out.loc[j, 'Award Year'] = 0
        out.loc[j, 'Oscar'] = 0
        out.loc[j, 'Cannes'] = 0
        out.loc[j, 'Berlin'] = 0
        out.loc[j, 'Venice'] = 0
        out.loc[j, 'Golden Globe'] = 0
        out.loc[j, 'Actor'] = 0
        out.loc[j, 'Producer'] = 0
        out.loc[j, 'Director'] = 0
        out.loc[j, 'Writer'] = 0
        out.loc[j, 'Budget'] = 0
        out.loc[j, 'Box Office'] = 0 
        
        try:
            out.loc[j, 'Birthdate'] = person['birth date']
        except KeyError:
            continue
        
        try:
            out.loc[j, 'Deathdate'] = person['death date']
        except KeyError:
            continue
        
        try: 
            out.loc[j, 'Title'] = tmp['title']
        except KeyError:
            continue
        
        try:
            out.loc[j, 'Type'] = tmp['kind']
        except KeyError:
            continue
        
        try:
            out.loc[j, 'Year'] = tmp['year']
        except KeyError:
            continue
        
        try:
            out.loc[j, 'IMDB Rating'] = tmp['rating']
        except KeyError:
            continue            
        
        try:
            out.loc[j, 'Running Time'] = tmp['runtimes'][0]
        except KeyError:
            continue
        
        try:
            for i in range(len(tmp['cast'])):
                if p in tmp['cast'][i]['name']: 
                    out.loc[j, 'Actor'] = 1
 
        except KeyError:
            continue
        
        try:
            for i in range(len(tmp['producer'])):
                if p in tmp['producer'][0]['name']:
                    out.loc[j, 'Producer'] = 1

        except KeyError:
            continue
        
        try:
            for i in range(len(tmp['director'])):    
                if p in tmp['director'][0]['name']:
                    out.loc[j, 'Director'] = 1
        except KeyError:
            continue
        
        try:
            for i in range(len(tmp['writer'])):
                if p in tmp['writer'][0]['name']:
                    out.loc[j, 'Writer'] = 1
        
        except KeyError:
            continue

        j += 1

In [4]:
out

,Name,Birthdate,Deathdate,Title,Type,Year,Award Year,Oscar,Cannes,Berlin,Venice,Golden Globe,IMDB Rating,Actor,Producer,Director,Writer,Budget,Box Office,Running Time


In [89]:
out.to_csv('./out.csv') # 실행한 것 까지 저장

In [102]:
df_index.loc[df_index.Actor==p] # 어디서 오류 났는지 점검 후 다시 인덱스 어디부터 실행시킬지 설정하기 

,Year,Actor,Role(s),Film,Ref.,Unnamed: 5,id
234,2020/21[103] (93rd),Riz Ahmed,Ruben Stone,Sound of Metal,[104],NaN,nm1981893


In [91]:
end_idx = df_index.loc[df_index.Actor==p].index

df_index.iloc[end_idx[0]]

Year          2020/21[103] (93rd)
Actor                 Steven Yeun
Role(s)                  Jacob Yi
Film                       Minari
Ref.                        [104]
Unnamed: 5                    NaN
id                      nm3081796
Name: 236, dtype: object

In [85]:
# 122, 133, 134, 180, 227, 228
end_idx[0]

228

In [94]:
df_index.iloc[227:235]

,Year,Actor,Role(s),Film,Ref.,Unnamed: 5,id
227,2017 (90th),Timoth_e Chalamet,Elio Perlman,Call Me by Your Name,[100],NaN,nm3154303
228,2017 (90th),Daniel Kaluuya,Chris Washington,Get Out,[100],NaN,nm2257207
229,2018 (91st),Rami Malek,Freddie Mercury,Bohemian Rhapsody,[101],NaN,nm1785339
230,2018 (91st),Willem Dafoe,Vincent van Gogh,At Eternity's Gate,[101],NaN,nm0000353
231,2019 (92nd),Antonio Banderas,Salvador Mallo,Pain and Glory,[102],NaN,nm0000104
232,2019 (92nd),Adam Driver,Charlie Barber,Marriage Story,[102],NaN,nm3485845
233,2019 (92nd),Jonathan Pryce,Cardinal Jorge Mario Bergoglio,The Two Popes,[102],NaN,nm0000596
234,2020/21[103] (93rd),Riz Ahmed,Ruben Stone,Sound of Metal,[104],NaN,nm1981893
